In [3]:
from pynq import Overlay, allocate
import numpy as np
import time

# 1. Load the overlay
overlay = Overlay("fe.bit")
overlay.download()

# 2. Reference to FIFO IPs
fifo_in = overlay.axi_fifo_in  # change if names differ
fifo_out = overlay.axi_fifo_out

# 3. Generate dummy packet data (64-bit = 8 bytes per word)
packet_count = 100
packet_length_bytes = 40  # each packet
words_per_packet = packet_length_bytes // 8

# Create stream of 100 packets, each with 5x 64-bit words
input_data = []
for i in range(packet_count):
    pkt = np.random.randint(0, 2**64, size=words_per_packet, dtype=np.uint64)
    input_data.extend(pkt)

input_buffer = allocate(shape=(len(input_data),), dtype=np.uint64)
input_buffer[:] = input_data

# 4. Send data into AXI-Stream FIFO
print("[INFO] Sending packets to feature extractor...")
fifo_in.write(input_buffer)

# 5. Wait briefly for processing
time.sleep(0.5)

# 6. Read output from AXI-Stream FIFO
# Assuming 1 feature per packet as 32-bit value
output_buffer = allocate(shape=(packet_count,), dtype=np.uint32)
fifo_out.read(output_buffer)

# 7. Display features
print("[INFO] Received features:")
for i, feat in enumerate(output_buffer):
    print(f"Packet {i+1:03d}: Feature = {feat}")

# 8. Cleanup
input_buffer.freebuffer()
output_buffer.freebuffer()


AttributeError: Could not find IP or hierarchy axi_fifo_in in overlay

In [2]:
from pynq import Overlay
overlay = Overlay("fe_h.bit")
overlay.download()
print(overlay.ip_dict.keys())


RuntimeError: Unable to parse metadata